In [1]:
import pandas as pd
import pymysql
import requests
import time

In [2]:
conn = pymysql.connect(host = 'localhost',
                       port=3306,
                       user = 'root',
                       password = 'rootpass',
                       db = 'jeju')

In [3]:
#수기로 수정한 contentsid 값 다시 적용
def change_value(df,changedict):
    if df['name'] in changedict:
        return changedict.get(df['name'])
    else:
        return df['contentsid'][0]

In [4]:
#sql에 저장된 이름으로 비짓제주에서의 아이디값 찾기
def find_contentsid(name):
    word_length = len(name)
    for i in range(1,word_length):
        search_word = name[:-i]
        result = df[df['title'].str.startswith(search_word)].loc[:,'contentsid'].values
        if len(result) == 0:
            continue
        elif len(result) == 1:
            print(f'{name}의 contentsid발견')
            return result
        else:
            print(f'{name}의 contesid확인 필요')
            return result
            

In [5]:
sql_input = "SELECT * FROM jeju_data_web where contentscd = '관광지'"
sql_df = pd.read_sql_query(sql_input, conn)

C:\Users\TJ\.conda\envs\py38\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [6]:
drop_lsts = ["isSite","type","rCode","posExact","address","roadAddr","isCallLink","adult","endPageUrl","mobileEndPageUrl","ktCallMd","ppc","imageModDate","displayCategory","category","categoryPaths","previewImages","streetPanorama","skyPanorama","insidePanorama","interiorPanorama","indoorPanorama","poiInfo","entranceCoords","theme","petrolInfo","marker","markerSelected","datalab","dynamicData","markerLabel","isParkingSupported"]

In [7]:
df =pd.read_csv('../jeju_tour_places_modified.csv')

In [8]:
df.columns

Index(['alltag', 'contentsid', 'contentscd', 'title', 'address', 'roadaddress',
       'tag', 'introduction', 'readcnt', 'likecnt', 'reviewcnt', 'markcnt',
       'snssharecnt', 'schedulecnt', 'visitcnt', 'evelpt', 'latitude',
       'longitude', 'phoneno', 'sbst', 'PHOTOSHARECOUNT', 'cate2cd', 'cate3cd',
       'imgpath', 'thumbnailpath'],
      dtype='object')

In [9]:
df.head(2)

,alltag,contentsid,contentscd,title,address,roadaddress,tag,introduction,readcnt,likecnt,...,evelpt,latitude,longitude,phoneno,sbst,PHOTOSHARECOUNT,cate2cd,cate3cd,imgpath,thumbnailpath
0,"휴식/힐링,체험관광,액티비티,사계절",CNTS_200000000012469,관광지,제로스테이션(제로포인트트레일),제주특별자치도 제주시 건입동 1136 2층,제주특별자치도 제주시 산지로 25 2층,"휴식/힐링,체험관광,액티비티,사계절,체험,레저/체험,어트랙션",제로스테이션은 산지천에 위치한 카페이자 길과 여행을 사랑하는 사람들을 위한 공간이다...,530,0,...,0.0,33.514733,126.529280,064-702-1947,"트레킹 프로그램과 관련해 사전 상담과 당일 모니터링, 완주 인증 기록 게시 등 다양...",0.0,레저/체험,그외,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...
1,"부모,커플,친구,경관/포토,휴식/힐링,테마공원,오름,사계절,청년,중/장년,휴식/치유...",CNTS_200000000009501,관광지,궁대오름(궁대악),제주특별자치도 서귀포시 성산읍 수산리 4711-1,제주특별자치도 서귀포시 성산읍 금백조로 446,"부모,커플,친구,경관/포토,휴식/힐링,테마공원,오름,사계절,청년,중/장년,휴식/치유...","오름의 모양새가 마치 활처럼 생기고, 오름 허리에 활(궁·弓)자 모양의 띠가 둘러져...",1723,0,...,0.0,33.440562,126.822089,064-787-4711,"오름의 모양새가 마치 활처럼 생기고, 오름 허리에 활(궁·弓)자 모양의 띠가 둘러져...",0.0,자연,오름,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103 entries, 0 to 1102
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   alltag           1103 non-null   object 
 1   contentsid       1103 non-null   object 
 2   contentscd       1103 non-null   object 
 3   title            1103 non-null   object 
 4   address          1099 non-null   object 
 5   roadaddress      1095 non-null   object 
 6   tag              1103 non-null   object 
 7   introduction     1101 non-null   object 
 8   readcnt          1103 non-null   int64  
 9   likecnt          1103 non-null   int64  
 10  reviewcnt        1103 non-null   int64  
 11  markcnt          1103 non-null   int64  
 12  snssharecnt      1103 non-null   int64  
 13  schedulecnt      1103 non-null   int64  
 14  visitcnt         1103 non-null   int64  
 15  evelpt           793 non-null    float64
 16  latitude         1094 non-null   float64
 17  longitude     

In [11]:
df[df['title'].str.startswith(sql_df['name'][0][:-6])].loc[:,'contentsid'].values

array(['CONT_000000000500457'], dtype=object)

In [12]:
testr=find_contentsid(sql_df['name'][0])

오설록 티 뮤지엄의 contentsid발견


In [13]:
testr

array(['CONT_000000000500457'], dtype=object)

In [14]:
sql_df['name'][0]

'오설록 티 뮤지엄'

In [16]:
sql_df.head()

,id,name,x,y,fullRoadAddress,phone,description,keywords,imageURL,categories,bizhourInfo,menus,menuImages,reviewCount,contentscd
0,11612628,오설록 티 뮤지엄,126.289393,33.306084,제주특별자치도 서귀포시 안덕면 서광리 1235-1 오설록,064-794-5312,자연을 생각하는 제주 오설록 티뮤지엄! |제주 서광 차밭과 맞닿아 있는 오설록 티뮤...,"녹차밀크스프레드, 롤케이크, 산지녹차, 덖음차, 녹차아이스크림, 주차가능",https://ldb-phinf.pstatic.net/20220110_254/164...,박물관,매일 09:00~18:00,"녹차와플오프레도 : 8,000원 / 녹차 아이스크림 : 5,000원 / 녹차 롤 케...",,11847,관광지
1,12047800,휴애리자연생활공원,126.635477,33.309436,제주특별자치도 서귀포시 남원읍 신례리 2081,064-732-2114,휴애리는 서귀포시 남원읍 신례리 한라산 자락에 있는 자연생활 체험 공원입니다. |2...,"동백축제, 감귤체험, 핑크뮬리축제, 제주동백축제, 제주도동백축제, 주차가능",https://ldb-phinf.pstatic.net/20220726_53/1658...,자연공원,"매일 09:00~18:00 연중무휴, 하절기(4월~9월) 입장마감 17:30 , 동...","성인 : 13,000원 / 청소년 : 11,000원 / 어린이 : 10,000원 /...",,6743,관광지
2,17052534,새연교,126.559029,33.238444,제주특별자치도 서귀포시 서홍동,064-760-3471,"서귀포항과 새섬을 잇는 다리입니다. |제주 전통배 테우의 모습을 옮겨 만들었으며, ...",산책로,https://ldb-phinf.pstatic.net/20150206_5/14232...,지역명소,,,,2481,관광지
3,11491630,정방폭포,126.573050,33.244748,제주특별자치도 서귀포시 동홍동 278,064-733-1530,천지연폭포·천제연폭포와 더불어 제주도 3대 폭포 중의 하나로서 정방하폭이라고도 한다...,소재경승지,https://ldb-phinf.pstatic.net/20150831_217/144...,폭포,매일 09:00~17:50 일몰시간에 따라 변경가능,"어른(개인) : 2,000원 / 군인 (하사이하) : 1,000원 / 어린이 아동...",,4316,관광지
4,11608192,만장굴,126.771619,33.528351,제주특별자치도 제주시 구좌읍 월정리 산41-5,064-710-7903,1962년 12월 3일 김녕사굴과 함께 천연기념물 제98호로 지정되었다. |총길이 ...,용암동굴,https://ldb-phinf.pstatic.net/20150831_156/144...,문화재,매일 09:00~18:00 입장시간 09:00-17:00 | 수요일 휴무 매월 첫째...,"어른 : 4,000원 / 청소년 : 2,000원 / 군인 : 2,000원 / 어린이...",,2897,관광지


In [15]:
namelsts = sql_df['name']

In [118]:
contentsid_lsts = []
for name in namelsts:
    tmp_dict={'name':name}
    tmp_result=find_contentsid(name)
    if len(tmp_result) == 1:
        tmp_dict['contentsid']=tmp_result
    else:
        tmp_dict['needcheck'] = True
        tmp_dict['contentsid'] = tmp_result
    contentsid_lsts.append(tmp_dict)
    print(f'{name} 검색 완료')

오설록 티 뮤지엄의 contentsid발견
오설록 티 뮤지엄 검색 완료
휴애리자연생활공원의 contentsid발견
휴애리자연생활공원 검색 완료
새연교의 contentsid발견
새연교 검색 완료
정방폭포의 contentsid발견
정방폭포 검색 완료
만장굴의 contentsid발견
만장굴 검색 완료
한림공원의 contentsid발견
한림공원 검색 완료
에코랜드 테마파크의 contentsid발견
에코랜드 테마파크 검색 완료
노형수퍼마켙의 contentsid발견
노형수퍼마켙 검색 완료
세계자동차&피아노 박물관의 contentsid발견
세계자동차&피아노 박물관 검색 완료
9.81 파크의 contentsid발견
9.81 파크 검색 완료
산굼부리의 contentsid발견
산굼부리 검색 완료
산방산의 contesid확인 필요
산방산 검색 완료
스누피가든의 contentsid발견
스누피가든 검색 완료
카멜리아힐의 contentsid발견
카멜리아힐 검색 완료
김녕해수욕장의 contesid확인 필요
김녕해수욕장 검색 완료
월정리해수욕장의 contesid확인 필요
월정리해수욕장 검색 완료
함덕해수욕장의 contentsid발견
함덕해수욕장 검색 완료
세화해수욕장의 contesid확인 필요
세화해수욕장 검색 완료
용두암의 contesid확인 필요
용두암 검색 완료
도두동무지개해안도로의 contentsid발견
도두동무지개해안도로 검색 완료
우도의 contesid확인 필요
우도 검색 완료
이호테우해수욕장의 contentsid발견
이호테우해수욕장 검색 완료
성산항의 contesid확인 필요
성산항 검색 완료
하귀애월해안도로의 contesid확인 필요
하귀애월해안도로 검색 완료
한라수목원의 contentsid발견
한라수목원 검색 완료
용눈이오름의 contentsid발견
용눈이오름 검색 완료
성산일출봉의 contentsid발견
성산일출봉 검색 완료
광치기해변의 contentsid발견
광치기해변 검색 완료
곽지해수욕장의 contesid확인 필요
곽지해수욕장 검색 완료
섭지코지의 contentsi

In [172]:
contentsid_df=pd.DataFrame(contentsid_lsts)

In [173]:
contentsid_df['needcheck'].fillna(False,inplace=True)

In [167]:
change_kv = {
    '마라도' : 'CONT_000000000500171',
'가파도' : 'CNTS_000000000018432',
'송악산' : 'CONT_000000000500378',
'천제연폭포': 'CNTS_000000000019331',
'쇠소깍' : 'CONT_000000000500384',
'금오름' : 'CNTS_000000000019460',
'한라산' : 'CONT_000000000500685',
'곽지해수욕장': 'CONT_000000000500056',
'하귀애월해안도로': 'CONT_000000000500425',
'성산항' : 'CNTS_000000000000978',
'우도' : 'CNTS_000000000018956',
'용두암' : 'CONT_000000000500469',
'세화해수욕장': 'CONT_000000000500361',
'월정리해수욕장': 'CONT_000000000500496',
'김녕해수욕장' : 'CONT_000000000500083',
'산방산' : 'CONT_000000000500285'
}

In [174]:
contentsid_df['contentsid']=contentsid_df.apply(lambda x: change_value(x,change_kv),axis=1)

In [176]:
contentsid_df['top3percent'] = True

In [178]:
contentsid_df.drop('needcheck',axis=1,inplace=True)

In [185]:
merge_df=pd.merge(left=df,right=contentsid_df,how='left',on='contentsid')

In [188]:
merge_df['top3percent'].fillna(False,inplace=True)

In [195]:
merge_df.head()

,alltag,contentsid,contentscd,title,address,roadaddress,tag,introduction,readcnt,likecnt,...,longitude,phoneno,sbst,PHOTOSHARECOUNT,cate2cd,cate3cd,imgpath,thumbnailpath,name,top3percent
0,"휴식/힐링,체험관광,액티비티,사계절",CNTS_200000000012469,관광지,제로스테이션(제로포인트트레일),제주특별자치도 제주시 건입동 1136 2층,제주특별자치도 제주시 산지로 25 2층,"휴식/힐링,체험관광,액티비티,사계절,체험,레저/체험,어트랙션",제로스테이션은 산지천에 위치한 카페이자 길과 여행을 사랑하는 사람들을 위한 공간이다...,530,0,...,126.529280,064-702-1947,"트레킹 프로그램과 관련해 사전 상담과 당일 모니터링, 완주 인증 기록 게시 등 다양...",0.0,레저/체험,그외,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...,NaN,False
1,"부모,커플,친구,경관/포토,휴식/힐링,테마공원,오름,사계절,청년,중/장년,휴식/치유...",CNTS_200000000009501,관광지,궁대오름(궁대악),제주특별자치도 서귀포시 성산읍 수산리 4711-1,제주특별자치도 서귀포시 성산읍 금백조로 446,"부모,커플,친구,경관/포토,휴식/힐링,테마공원,오름,사계절,청년,중/장년,휴식/치유...","오름의 모양새가 마치 활처럼 생기고, 오름 허리에 활(궁·弓)자 모양의 띠가 둘러져...",1723,0,...,126.822089,064-787-4711,"오름의 모양새가 마치 활처럼 생기고, 오름 허리에 활(궁·弓)자 모양의 띠가 둘러져...",0.0,자연,오름,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...,NaN,False
2,"커플,부모,혼자,친구,아이,맑음,흐림,비.눈,경관/포토,휴식/힐링,미술/박물관,사계...",CNTS_200000000012075,관광지,헌마공신김만일기념관,서귀포시 남원읍 서성로 919,서귀포시 남원읍 서성로 919,"커플,부모,혼자,친구,아이,맑음,흐림,비.눈,경관/포토,휴식/힐링,미술/박물관,사계...",임진왜란 당시 전쟁에 사용할 말 500마리를 헌납하면서 국난극복에 기여한 김만일(1...,323,2,...,NaN,064-805-9801,NaN,0.0,문화관광,전시관,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...,NaN,False
3,관광지,CNTS_200000000013590,관광지,본초족욕,제주특별자치도 제주시 연동 1326,제주특별자치도 제주시 은수길 65,"관광지,안전여행스탬프,experience",천연 족욕 테라피를 즐기면서 심신의 피로를 확 날려버릴 수 있는 곳,40,1,...,126.488172,010249938913,NaN,0.0,레저/체험,그외,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...,NaN,False
4,"실내,문화유적지,가을,문화관광,역사유적,어트랙션,공용주차장,어려움",CNTS_000000000021292,관광지,강병대교회,제주특별자치도 서귀포시 대정읍 상모리 3846,제주특별자치도 서귀포시 대정읍 상모대서로 43-3,"실내,문화유적지,가을,문화관광,역사유적,어트랙션",우리나라에서 가장 오래된 군인 교회,2839,0,...,126.257220,070-7790-3860,건립 당시의 원형이 절반 이상 남아있는 생생한 역사의 흔적\r\n서귀포시 대정읍에 ...,NaN,문화관광,문화재,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...,NaN,False


In [210]:
get_image_df=merge_df['imgpath']

In [212]:
for i,v in enumerate(get_image_df):
    tmp_name = merge_df['contentsid'][i]
    url = v
    file_path = '../tour_images/' + tmp_name + '.jpg'
    time.sleep(1)
    response = requests.get(url)
    if response.status_code == 200:
        with open(file_path,'wb') as f:
            f.write(response.content)
            print(f'{i}번째 {tmp_name} 저장완료')
    else:
        print('response확인 요망 응답실패')

0번째 CNTS_200000000012469 저장완료
1번째 CNTS_200000000009501 저장완료
2번째 CNTS_200000000012075 저장완료
3번째 CNTS_200000000013590 저장완료
4번째 CNTS_000000000021292 저장완료
5번째 CONT_000000000500653 저장완료
6번째 CONT_000000000500194 저장완료
7번째 CONT_000000000500026 저장완료
8번째 CNTS_000000000020935 저장완료
9번째 CONT_000000000500138 저장완료
10번째 CNTS_000000000022844 저장완료
11번째 CNTS_000000000022459 저장완료
12번째 CONT_000000000500193 저장완료
13번째 CNTS_000000000021401 저장완료
14번째 CONT_000000000500178 저장완료
15번째 CNTS_000000000021612 저장완료
16번째 CNTS_000000000018460 저장완료
17번째 CONT_000000000500013 저장완료
18번째 CNTS_000000000020170 저장완료
19번째 CONT_000000000500327 저장완료
20번째 CNTS_000000000022624 저장완료
21번째 CNTS_200000000013530 저장완료
22번째 CNTS_000000000022531 저장완료
23번째 CNTS_200000000013306 저장완료
24번째 CONT_000000000500215 저장완료
25번째 CONT_000000000500579 저장완료
26번째 CNTS_200000000010815 저장완료
27번째 CONT_000000000500460 저장완료
28번째 CONT_000000000500050 저장완료
29번째 CNTS_000000000020560 저장완료
30번째 CNTS_200000000007279 저장완료
31번째 CNTS_000000000020936 저장완료
32번째 CNTS_20000000

ChunkedEncodingError: ("Connection broken: ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None)", ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))

In [200]:
resp = requests.get(merge_df['imgpath'][0])

In [204]:
with open('../tour_images/test.jpg','wb') as f:
    f.write(resp.content)

In [213]:
url

'https://api.cdn.visitjeju.net/photomng/imgpath/201804/30/b32d8cc2-d0ea-4271-a210-624d8dde3dd3.jpg'